## dataset handler (folder-based)

In [2]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch
import config as cfg
from models.lenet import LeNet5

from data_loader.dataset import dset_imageFolder

In [3]:
# Load Dataset from folders
train_dataset, valid_dataset,test_dataset = dset_imageFolder()

train_loader = DataLoader(dataset=train_dataset, batch_size=cfg.batch_size, shuffle=cfg.shuffle, num_workers=cfg.num_workers)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=cfg.batch_size, shuffle=cfg.shuffle, num_workers=cfg.num_workers)
test_loader = DataLoader(dataset=test_dataset, batch_size=cfg.batch_size, shuffle=cfg.shuffle, num_workers=cfg.num_workers)

In [4]:
# create train and valid dataset using train_dataset and random split
# len_train = round(len(train_dataset) * 0.7)
# len_valid = len(train_dataset) - len_train

# train_dataset, valid_dataset = random_split(train_dataset, 
#                                             lengths=[len_train, len_valid], 
#                                             generator=torch.Generator().manual_seed(1))

In [6]:
# demo for handler and loader
data, target = next(iter(train_loader)) 

print(data.shape)

print(target.shape)

torch.Size([5, 3, 128, 128])
torch.Size([5])


In [7]:
model = LeNet5(n_classes=cfg.n_classes)
print(model)

LeNet5(
  (convolutions): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=75000, out_features=84, bias=True)
    (1): Tanh()
    (2): Linear(in_features=84, out_features=3, bias=True)
  )
)


In [9]:
# sample output
output = model(data)
print(output)

def accuracy(target, prediction):
    with torch.no_grad():
        pred = torch.argmax(prediction, dim=1)
        assert pred.shape[0] == len(target)
        correct = 0
        correct += torch.sum(pred == target).item()
        acc = correct / len(target)
    return acc

tensor([[0.3368, 0.3316, 0.3316],
        [0.3446, 0.3272, 0.3282],
        [0.3415, 0.3372, 0.3213],
        [0.3385, 0.3286, 0.3329],
        [0.3382, 0.3311, 0.3307]], grad_fn=<SoftmaxBackward0>)


In [11]:
accuracy(target, output) * 100

20.0

In [9]:
# device, cuda or cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# loss function
loss_fn = torch.nn.CrossEntropyLoss()

# optimization algorithm
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.train(True)

LeNet5(
  (convolutions): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=75000, out_features=84, bias=True)
    (1): Tanh()
    (2): Linear(in_features=84, out_features=3, bias=True)
  )
)

In [11]:
# implement training process
# use tqdm for progress bar
from tqdm.notebook import tqdm
# epoch loop
epochs = 5

# training epoch
for epoch in tqdm(range(0, epochs)):
    
    # training batch 
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):    

        optimizer.zero_grad() # remove gradients from previous loop
        
        outputs = model(inputs) # classify inputs
        
        loss = loss_fn(outputs, targets) # calculate loss between outputs and targets
        
        loss.backward() # implement backpropagation
        
        optimizer.step() # update weights/parameters
        
        break
    
    # run validate model with validation data
    # model.train(False)
    model.eval()
    for batch_idx, (inputs, targets) in enumerate(valid_loader):
        break

  0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
print(outputs[0])

tensor([0.4681, 0.3260, 0.2060], grad_fn=<SelectBackward0>)


In [ ]:
# implement data/, loss/, models/, optimizer/, scheduler/, tools/ or utilities/

# implement train.py

# implement test.py
